In [38]:
# 0. 기본 설정 (제공해주신 성별 파이프라인과 동일한 스타일)
import kfp, uuid, os
print("KFP:", kfp.__version__)

from kfp import dsl
from kfp.dsl import component, pipeline
from kfp import kubernetes as k8s   # ← 이게 핵심!

# 공통 변수
NAMESPACE = os.environ['NB_PREFIX'].split('/')[2]
PVC_NAME   = f"fmnist-pvc-{uuid.uuid1().hex[:8]}"
MODEL_NAME = f"torch-model-{uuid.uuid1().hex[:8]}"
ISVC_NAME  = f"torchserve-{uuid.uuid1().hex[:8]}"
EPOCHS     = 3

print(f"PVC: {PVC_NAME} | Model: {MODEL_NAME} | ISVC: {ISVC_NAME}")


KFP: 2.11.0
PVC: fmnist-pvc-1831fc9a | Model: torch-model-1831fe0c | ISVC: torchserve-1831fef2


In [39]:
# 1. PVC 미리 생성 (노트북에서 한 번만 실행)
import subprocess, textwrap
subprocess.run(["kubectl", "apply", "-f", "-"], input=textwrap.dedent(f"""
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: {PVC_NAME}
  namespace: {NAMESPACE}
spec:
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 10Gi
"""), text=True, check=True)
print("PVC 생성 완료!")

persistentvolumeclaim/fmnist-pvc-1831fc9a created
PVC 생성 완료!


In [40]:
from kfp.dsl import component
import os

@component(
    base_image="python:3.11-slim",
    packages_to_install=["requests"]
)
def download_fmnist(pvc_path: str) -> str:
    """
    Fashion MNIST 데이터셋 4개 파일을 다운로드하고 PVC에 압축 해제합니다.
    반환값: PVC 내부 경로 (예: "/mnt/pvc")
    """
    import os
    import requests
    import gzip
    from io import BytesIO

    # 다운로드할 4개 파일
    urls = [
        ("https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz", "train-images-idx3-ubyte"),
        ("https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-labels-idx1-ubyte.gz", "train-labels-idx1-ubyte"),
        ("https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz",  "t10k-images-idx3-ubyte"),
        ("https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz",  "t10k-labels-idx1-ubyte"),
    ]

    os.makedirs(pvc_path, exist_ok=True)

    def download_and_extract(url: str, filename: str):
        print(f"다운로드 중: {url}")
        response = requests.get(url, timeout=60)
        response.raise_for_status()

        gz_path = os.path.join(pvc_path, f"{filename}.gz")
        raw_path = os.path.join(pvc_path, filename)

        # .gz 저장
        with open(gz_path, "wb") as f:
            f.write(response.content)

        # 압축 해제 후 .gz 삭제
        with gzip.open(gz_path, "rb") as fin, open(raw_path, "wb") as fout:
            fout.write(fin.read())
        os.remove(gz_path)
        print(f"완료: {raw_path}")

    for url, name in urls:
        download_and_extract(url, name)

    print(f"Fashion MNIST 데이터셋이 {pvc_path} 에 준비되었습니다.")
    return pvc_path

In [41]:
from kfp.dsl import component
import os

@component(
    base_image="pytorch/pytorch:2.3.0-cuda11.8-cudnn8-runtime",
    packages_to_install=["torchvision", "pillow", "numpy"]
)
def train_fmnist(
    pvc_path: str,
    model_name: str,
    epochs: int = 3
) -> str:
    """
    Fashion MNIST 학습 + TorchServe용 handler.py + config.properties 생성
    반환값: 모델 디렉터리 경로 (예: "/mnt/pvc/fmnist_model")
    """
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import Dataset, DataLoader
    import torchvision.transforms as T
    import numpy as np
    import json
    from pathlib import Path

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # ------------------- 1. Dataset -------------------
    class FashionMNISTRaw(Dataset):
        def __init__(self, img_path, lbl_path, transform=None):
            with open(img_path, "rb") as f:
                data = f.read()
                self.images = np.frombuffer(data, np.uint8, offset=16).reshape(-1, 28, 28)
            with open(lbl_path, "rb") as f:
                data = f.read()
                self.labels = np.frombuffer(data, np.uint8, offset=8)
            self.transform = transform

        def __len__(self):
            return len(self.labels)

        def __getitem__(self, idx):
            img = self.images[idx].astype(np.float32)
            img = torch.from_numpy(img).unsqueeze(0) / 255.0   # (1,28,28)
            if self.transform:
                img = self.transform(img)
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return img, label

    transform = T.Compose([T.Normalize((0.5,), (0.5,))])

    train_dataset = FashionMNISTRaw(
        f"{pvc_path}/train-images-idx3-ubyte",
        f"{pvc_path}/train-labels-idx1-ubyte",
        transform=transform
    )
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

    # ------------------- 2. Model -------------------
    class Net(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
            self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
            self.pool  = nn.MaxPool2d(2, 2)
            self.fc1   = nn.Linear(64 * 7 * 7, 128)
            self.fc2   = nn.Linear(128, 10)

        def forward(self, x):
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            x = x.view(-1, 64 * 7 * 7)
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    model = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # ------------------- 3. Training -------------------
    model.train()
    for epoch in range(1, epochs + 1):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 1):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 0:
                print(f"Epoch {epoch}/{epochs} | Batch {i} | Loss: {running_loss/100:.4f}")
                running_loss = 0.0
    print("Training 완료!")

    # ------------------- 4. Save model -------------------
    model_dir = f"{pvc_path}/fmnist_model"
    os.makedirs(model_dir, exist_ok=True)
    torch.save(model.state_dict(), f"{model_dir}/fmnist_cnn.pth")
    print(f"모델 저장: {model_dir}/fmnist_cnn.pth")

    # ------------------- 5. handler.py -------------------
    handler_code = """\
from ts.torch_handler.base_handler import BaseHandler
import torch
import torchvision.transforms as transforms
from PIL import Image
import io
import base64
import logging

logger = logging.getLogger(__name__)

class ImageClassifierHandler(BaseHandler):
    def __init__(self):
        super().__init__()
        self.transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((28, 28)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def preprocess(self, data):
        image = data[0].get("data") or data[0].get("body")
        if isinstance(image, str):
            image = base64.b64decode(image)
        image = Image.open(io.BytesIO(image))
        return self.transform(image).unsqueeze(0).to(self.device)

    def inference(self, inputs):
        with torch.no_grad():
            outputs = self.model(inputs)
            _, predicted = torch.max(outputs, 1)
        return predicted

    def postprocess(self, inference_output):
        return [int(inference_output[0])]
"""
    Path(f"{model_dir}/handler.py").write_text(handler_code)
    print("handler.py 생성 완료")

    # ------------------- 6. config.properties -------------------
    # f-string 안에 { } 가 들어가면 {{ }} 로 이스케이프
    import json

    snapshot = {
        "name": "startup.cfg",
        "modelCount": 1,
        "models": {
            model_name: {
                "1.0": {
                    "defaultVersion": True,
                    "marName": f"{model_name}.mar",
                    "minWorkers": 1
                }
            }
        }
    }
    
    config_content = (
        "inference_address=http://0.0.0.0:8085\n"
        "management_address=http://0.0.0.0:8085\n"
        "metrics_address=http://0.0.0.0:8082\n"
        "model_store=/mnt/pvc/fmnist_model/model-store\n"
        f"model_snapshot={json.dumps(snapshot)}\n"
    )


    config_dir = f"{model_dir}/config"
    os.makedirs(config_dir, exist_ok=True)
    Path(f"{config_dir}/config.properties").write_text(config_content)
    print("config.properties 생성 완료")

    return model_dir

In [42]:
from kfp.dsl import component

@component(
    base_image="pytorch/pytorch:2.3.0-cuda11.8-cudnn8-runtime",
    packages_to_install=["torch-model-archiver", "torchserve"]
)
def create_mar(model_dir: str, model_name: str) -> str:
    """
    TorchServe용 .mar 파일 생성 컴포넌트
    - model_dir: handler.py, *.pth, config.properties가 포함된 폴더
    - 반환값: 생성된 .mar 파일 경로
    """
    import os
    import subprocess

    model_store = os.path.join(model_dir, "model-store")
    os.makedirs(model_store, exist_ok=True)

    cmd = [
        "torch-model-archiver",
        "--model-name", model_name,
        "--version", "1.0",
        "--serialized-file", f"{model_dir}/fmnist_cnn.pth",
        "--handler", f"{model_dir}/handler.py",
        "--export-path", model_store,
        "--force"
    ]

    print("실행 중:", " ".join(cmd))
    subprocess.run(cmd, check=True)

    mar_path = f"{model_store}/{model_name}.mar"
    print("생성된 MAR 파일:", mar_path)
    return mar_path


In [43]:
from kfp.dsl import component, OutputPath

@component(
    base_image="python:3.11-slim",
    packages_to_install=["kubernetes", "pyyaml"]
)
def deploy_kserve(
    namespace: str,
    isvc_name: str,
    model_dir: str,
    pvc_name: str,
    isvc_yaml_out: OutputPath(str),
    isvc_url_out: OutputPath(str)
):
    """
    PVC에 저장된 PyTorch/TorchServe 모델을 KServe로 배포하는 컴포넌트.
    """
    import time, yaml
    from kubernetes import client, config

    # 1) K8s 연결
    try:
        config.load_incluster_config()
    except:
        config.load_kube_config()

    crd = client.CustomObjectsApi()

    # 2) PVC를 modelUri로 사용하는 predictor
    isvc_body = {
        "apiVersion": "serving.kserve.io/v1beta1",
        "kind": "InferenceService",
        "metadata": {
            "name": isvc_name,
            "namespace": namespace,
        },
        "spec": {
            "predictor": {
                "model": {
                    "modelFormat": { "name": "pytorch" },
                    "storageUri": f"pvc://{pvc_name}{model_dir}",
                    "resources": {
                        "requests": {"cpu": "200m", "memory": "1Gi"},
                        "limits":   {"cpu": "1",    "memory": "2Gi"}
                    }
                }
            }
        }
    }

    # 3) 만들거나 패치
    group = "serving.kserve.io"
    version = "v1beta1"
    plural = "inferenceservices"

    try:
        crd.create_namespaced_custom_object(group, version, namespace, plural, isvc_body)
        print("InferenceService 생성 완료")
    except client.ApiException as e:
        if e.status == 409:
            crd.patch_namespaced_custom_object(group, version, namespace, plural, isvc_name, isvc_body)
            print("기존 InferenceService 패치 완료")
        else:
            raise

    # 4) Ready 대기
    url = None
    for _ in range(60):
        obj = crd.get_namespaced_custom_object(group, version, namespace, plural, isvc_name)
        status = obj.get("status", {})
        address = status.get("address", {})
        url = address.get("url")
        if url:
            break
        time.sleep(3)

    # 5) 출력 저장
    with open(isvc_yaml_out, "w") as f:
        yaml.safe_dump(isvc_body, f)
    with open(isvc_url_out, "w") as f:
        f.write(url or "")

    print("배포 완료. URL =", url)


In [47]:
from kfp import dsl
from kfp import kubernetes as k8s

PVC_NAME   = "cpu-notebook-workspace"
NAMESPACE  = "kbm-u-kubeflow-tutorial"
MODEL_NAME = "fmnist"
ISVC_NAME  = "fmnist-pytorch"
EPOCHS     = 3

@dsl.pipeline(
    name="FashionMNIST → TorchServe → KServe",
    description="PVC에 FMNIST를 학습하고, 같은 PVC에서 KServe로 배포하는 파이프라인"
)
def fmnist_pipeline(
    pvc_name: str = PVC_NAME,
    model_name: str = MODEL_NAME,
    isvc_name: str = ISVC_NAME,
    namespace: str = NAMESPACE,
    epochs: int = EPOCHS,
):
    # 1) 데이터 다운로드
    dl = download_fmnist(pvc_path="/mnt/pvc")
    k8s.mount_pvc(dl, pvc_name=pvc_name, mount_path="/mnt/pvc")

    # 2) 학습 (다운로드 이후 실행 + GPU)
    train = train_fmnist(
        pvc_path="/mnt/pvc",
        model_name=model_name,
        epochs=epochs,
    )
    k8s.mount_pvc(train, pvc_name=pvc_name, mount_path="/mnt/pvc")
    train.add_node_selector_constraint("nvidia.com/gpu.present=true")
    train.set_gpu_limit(1)
    train.after(dl)  # 🔴 반드시 다운로드 이후에 학습 시작

    # 3) KServe 배포 (학습 이후 실행)
    # train_fmnist가 /mnt/pvc/fmnist_model 에 저장한다고 가정하면
    # PVC 루트 기준 경로는 /fmnist_model
    deploy = deploy_kserve(
        namespace=namespace,
        isvc_name=isvc_name,
        model_dir="/fmnist_model",
        pvc_name=pvc_name,
    )
    deploy.after(train)  # 🔴 학습이 끝난 뒤 배포 실행

    # 캐싱 끄기 (선택)
    for t in [dl, train, deploy]:
        t.set_caching_options(enable_caching=False)


In [48]:
from kfp import compiler

compiler.Compiler().compile(
    pipeline_func=fmnist_pipeline,
    package_path="fmnist-kserve-pipeline.yaml",
)

print("[Done] fmnist-kserve-pipeline.yaml created")


[Done] fmnist-kserve-pipeline.yaml created
